# 작업내용

## 전처리 추가
1.  괄호로 들어가는 보조 설명 제거   
 ex. [Music]
2. 구어에서 발생하는 filler words 제거
3. 반복단어 제거
4. 문장 끝점 추가(rpunct)



## 트랜스포머 모델 테스트
1. "facebook/bart-large-cnn"
  - 현재 탑재된 모델
  - input으로 받을 수 있는 분량이 4000자(character 숫자 기준) 미만임. 영어에서 대략 900~ 1000단어 정도로 a4용지 1장 정도 분량. 영상은 대략 4~5분 이하의 길이임

- 테스트 가능한 모델이 4개(해당 코랩 파일에 2개, 허깅페이스 api로 테스트 가능한 모델, 가연님이 제일 처음에 테스트 하셨던 모델) 
- 이 중에 이 코랩 파일에 있는 1번 모델이 우리가 탑재한 모델인데, 얘는 4000자 제한이 있음.
- 4000자 이하의 길이인 영상이 test를 하는 baseline이어야함.

영상을 3~4개를 선택해서(자동생성자막/매뉴얼자막, 4000자이하/이상)
4개의 모델에 각각 돌려보고, 그리고 추가로 전처리 과정(괄호제거, fillerwords제거, rpunct적용)을 전혀 거치지 않은 자막으로도 돌려보시고 결과를 따로 


In [1]:
!pip install youtube_transcript_api
!pip install pytube
!pip install transformers
!pip install rpunct

     |████████████████████████████████| 56 kB 3.5 MB/s 
     |████████████████████████████████| 3.8 MB 7.5 MB/s 
     |████████████████████████████████| 596 kB 54.1 MB/s 
     |████████████████████████████████| 895 kB 59.8 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 6.5 MB 58.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 981 kB 7.1 MB/s 
     |████████████████████████████████| 9.9 MB 46.2 MB/s 
     |████████████████████████████████| 213 kB 76.6 MB/s 
     |████████████████████████████████| 804.1 MB 2.5 kB/s 
     |████████████████████████████████| 125 kB 72.0 MB/s 
     |████████████████████████████████| 1.2 MB 55.1 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |████████████████████████████████| 1.7 MB 87.0 MB/s 
     |████████████████████████████████| 312 kB

In [1]:
import pandas as pd
import numpy as np
from youtube_transcript_api import YouTubeTranscriptApi as yta
from pytube import YouTube, extract
from transformers import pipeline
import re

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# 영상에서 자막 추출

In [2]:
# 링크 넣어서 자막 받아오는 함수 (돌려주는 건 합치지 않은 자막과 합친 자막)

def get_subtitle(youtube_link) :
  
  video_id = extract.video_id(youtube_link)
  subtitle = yta.get_transcript(video_id)
  
  subtitle_merge = ''
  for line in subtitle :
    subtitle_merge += line['text'] + " "
  
  return subtitle, subtitle_merge

In [38]:
# 자막 받아오고자 하는 유튜브 영상의 링크 넣기
subtitle, subtitle_merge = get_subtitle("https://www.youtube.com/watch?v=2B-iZGNiPA0")
# 자막 길이와 앞부분 내용 확인
print(len(subtitle_merge))
print(subtitle_merge)

5836
over the past two years you've likely taken a covet rapid antigen test and you probably know what to do swab swirl and wait for results a unique biochemical reaction involving saline tiny particles of gold and paper with antibodies is helping to give us quick and relatively accurate results i've never felt more attractive in my life okay three four mississippi five today kate is taking a covet test while explaining the biochemical reactions 15 mississippi i'm gonna go the other way okay now i'm gonna do the disgusting thing and stick this in the back of my throat okay okay that's all i can do all right an epidemiologist named michael mina he is strongly suggesting for this particular variant for omicron that you swab in the back of your throat and your nose if you're gonna do the rapid test and so we're just looking for the highest viral load some people are swabbing their nose and getting a negative and then they swab their throat and getting a positive and that is just supportin

In [4]:
# 자막 받아오기 - 4000자 이상 길이 (약 15분), 매뉴얼 자막
subtitle, subtitle_merge = get_subtitle("https://www.youtube.com/watch?v=uXrCeiQxWyc")


# 전처리



## 1. 괄호나 대괄호로 처리된 description이 있는 경우 제거

In [39]:
# 정규표현식으로 괄호 또는 대괄호 안에 있는 문자열 제거
parenthesis = "\(.*\)|\s-\s.*"
bracket = "\[.*\]|\s-\s.*"

parenthesis_cleaned_subtitle = re.sub(parenthesis,"", subtitle_merge)
bracket_cleaned_subtitle = re.sub(bracket, "", parenthesis_cleaned_subtitle)

## 2. 중복단어 반복 제거




In [40]:
bracket_cleaned_subtitle

"over the past two years you've likely taken a covet rapid antigen test and you probably know what to do swab swirl and wait for results a unique biochemical reaction involving saline tiny particles of gold and paper with antibodies is helping to give us quick and relatively accurate results i've never felt more attractive in my life okay three four mississippi five today kate is taking a covet test while explaining the biochemical reactions 15 mississippi i'm gonna go the other way okay now i'm gonna do the disgusting thing and stick this in the back of my throat okay okay that's all i can do all right an epidemiologist named michael mina he is strongly suggesting for this particular variant for omicron that you swab in the back of your throat and your nose if you're gonna do the rapid test and so we're just looking for the highest viral load some people are swabbing their nose and getting a negative and then they swab their throat and getting a positive and that is just supporting th

In [33]:
# bracket_cleaned_subtitle = "as as we watch watch spike that that we've you know know ah"

In [41]:
splited = bracket_cleaned_subtitle.split()

repeated_cleaned_subtitle = ""

for i in range(len(splited) -1) :
  if splited[i] == splited[i+1] :
    pass
  else :
    repeated_cleaned_subtitle += splited[i] + " "

repeated_cleaned_subtitle += splited[-1]




## 3. filler words 제거하기
- 만들어진 filler words 리스트가 없어서 직접 구글 검색 등을 통해 사용 빈도가 높은 filler words 들을 수집하여 list를 생성함


In [31]:
# 수집한 fillerwords 목록을 fillerwords 변수에 할당
# fillerwords = ["i mean", "basically", "you know", "umm", "um", "uh", "huh", "er", "eh", "ah", "like that", "just", "really", "somehow", "i guess", "i suppose", "like i said", "or something like that", "kind of", "sort of", "you see", "see what i mean", "yeah"]

In [8]:
# fw_list = []
# for i in fillerwords :
#   fw_list.append(" " + i + " ")
# fw_list = '|'.join(fw_list)
# fw_list

' i mean | basically | you know | umm | um | uh | huh | er | eh | ah | like that | just | really | somehow | i guess | i suppose | like i said | or something like that | kind of | sort of | you see | see what i mean | yeah '

In [43]:
# fw_cleaned_test에 클린이 된 text가 할당됨
fw_list = " i mean | basically | you know | umm | um | uh | huh | er | eh | ah | like that | just | really | somehow | i guess | i suppose | like i said | or something like that | kind of | sort of | you see | see what i mean | yeah "
fw_cleaned_text = re.sub(fw_list, " ", repeated_cleaned_subtitle)
fw_cleaned_text

"over the past two years you've likely taken a covet rapid antigen test and you probably know what to do swab swirl and wait for results a unique biochemical reaction involving saline tiny particles of gold and paper with antibodies is helping to give us quick and relatively accurate results i've never felt more attractive in my life okay three four mississippi five today kate is taking a covet test while explaining the biochemical reactions 15 mississippi i'm gonna go the other way okay now i'm gonna do the disgusting thing and stick this in the back of my throat okay that's all i can do all right an epidemiologist named michael mina he is strongly suggesting for this particular variant for omicron that you swab in the back of your throat and your nose if you're gonna do the rapid test and so we're looking for the highest viral load some people are swabbing their nose and getting a negative and then they swab their throat and getting a positive and that is supporting the fact that omi

## 4. rpunct사용한 전처리 - 문장 구별 및 온점 처리

https://huggingface.co/felflare/bert-restore-punctuation?text=My+name+is+Clara+and+I+live+in+Berkeley%2C+California.

In [44]:
from rpunct import RestorePuncts

In [45]:
rpunct = RestorePuncts()
cleaned_original_text = rpunct.punctuate(fw_cleaned_text)
print(len(cleaned_original_text))
print(cleaned_original_text[:1000])

5882
Over the past two years, you've likely taken a Covet Rapid Antigen test and you probably know what to do. Swab, swirl, and wait for results. A unique biochemical reaction involving saline, tiny particles of gold and paper with antibodies is helping to give us quick and relatively accurate results. I've never felt more attractive in my life. Okay, three, four, Mississippi Five. Today, Kate is taking a Covet test while explaining the biochemical reactions. 15 Mississippi. I'm gonna go the other way. Okay, now I'm gonna do the disgusting thing and stick this in the back of my throat. Okay, that's all I can do, all right. An epidemiologist named Michael Mina. He is strongly suggesting for this particular variant for Omicron that you swab in the back of your throat and your nose if you're gonna do the rapid test. And so we're looking for the highest viral load. Some people are swabbing their nose and getting a negative, and then they swab their throat and getting a positive. and that i

# 요약 모델 테스트

## 1. "facebook/bart-large-cnn" 모델 사용한 요약 생성
- 해당 모델의 경우 input으로 4000단어 미만밖에 받지 못함

In [46]:
# "facebook/bart-large-cnn"모델 사용해서 요약 생성하는 함수
def summarize(subtitle):
  summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
  summarized_subtitle = summarizer(subtitle, max_length=300, min_length=30, do_sample=False)

  return summarized_subtitle[0]["summary_text"]


In [ ]:
# "facebook/bart-large-cnn"모델 사용해서 요약 생성하기

# 문장구별 및 온점처리 없이 요약
#summarize(subtitle_merge)

In [47]:
# 문장구별 및 온점처리 후 요약
summarize(cleaned_original_text[:3999])

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

'A unique biochemical reaction involving saline, tiny particles of gold and paper with antibodies is helping to give us quick and relatively accurate results. Kate is taking a Covet test while explaining the biochemical reactions.'

## 2. "sshleifer/distilbart-cnn-12-6" 모델 테스트

- api로 테스트 가능 https://huggingface.co/sshleifer/distilbart-cnn-12-6

- long documentation 요약 
  - 튜토리얼 영상 https://www.youtube.com/watch?v=78KjuKYiF6s&t=228s
  - https://gist.github.com/saprativa/b5cb639e0c035876e0dd3c46e5a380fd


허깅페이스에서 api 이용해서 4000자 이하 자막 테스트 한 결과, 퀄이 좋지는 않음



'As we watch the spike that we've been seeing in commodity prices, we have heard a renewal of calls in the United States to drill more that the U.S. should be energy independent . Rick Newman has been looking into that question and it's not as easy as that I think is the is the bottom line .'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
# long Long Document Summarization using Hugging Face Transformers 

# wrap the output in colab cells
"""
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)
"""

In [ ]:
"# 요약할 문서 FileContent 변수에 할당하기
FileContent = cleaned_original_text
FileContent

'Value Creation. Wealth Creation. These are powerful words. Maybe You think of finance, you think of innovation, you think of creativity. But Who are the value creators? If We use that word, we must be implying that some people aren\'t creating value. Who are they? The Couch potatoes? The Value extractors? The Value destroyers? To Answer this question, we actually have to have a proper theory of value. And I\'m here as an economist to break it to you that we\'ve lost our way on this question. Now, don\'t look so surprised. What I Mean by that is,, we\'ve stopped contesting it. We\'ve stopped actually asking tough questions about what is the difference between value creation and value extraction, productive and unproductive activities. Now, let me give you some context here. 2009 was about a year and a half after one of the biggest financial crises of our time, second only to the 1929 Great Depression, and the CEO of Goldman Sachs said Goldman Sachs Workers are the most productive in th

In [ ]:
len(FileContent)

19519

In [ ]:
# Load the Model and Tokenizer
# import and initialize the tokenizer and model from the checkpoint

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

In [ ]:
# Convert entire document to sentences using 'nltk' 
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Create the chunks


# initialize

length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

5

In [ ]:
# With special tokens added
sum([len(tokenizer(c).input_ids) for c in chunks])


4096

In [ ]:
len(tokenizer(FileContent).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (4087 > 1024). Running this sequence through the model will result in indexing errors


4087

In [ ]:
# without special tokens added 
sum([len(tokenizer.tokenize(c)) for c in chunks])

4086

In [ ]:
len(tokenizer.tokenize(FileContent))

4085

In [ ]:
# Get the inputs
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
inputs

In [ ]:
# Output
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

 Goldman Sachs CEO said Goldman Sachs workers are the most productive in the world. Peter Bergen: We've stopped asking tough questions about what is the difference between value creation and value extraction, productive and unproductive activities. He argues that we have to have a proper theory of value to answer this question.
 Adam Smith argued that industrial labor was the source of the value that was getting siphoned out of the economy. Adam Smith showed that 10 specialized workers who had been invested in, in their human capital, could produce 4,800 pins a day, as opposed to one by an unspecialized worker. The big revolution that happened with the current system of economic thinking that we have, which is called "neoclassical Economics," was that the logic completely changed.
 Up until 1970, most of the financial sector was not even included in GDP. Instead of pausing and asking, "What is it actually doing?" -- that was a missed opportunity. This real focus on prices and also shar

## 3. "human-centered-summarization/financial-summarization-pegasus" 모델 테스트

- api 테스트 : https://huggingface.co/human-centered-summarization/financial-summarization-pegasus

- 결과
'Oil, coal, solar and wind are all forms of energy in the United States. `
